# Visual 4: Geographic Distribution of Median Housing Prices

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load housing data
# Automatically detect the correct file based on available columns
try:
    # Try main data.csv first
    df = pd.read_csv('../data files/data.csv')
    print(f"Loaded data.csv with {len(df)} records")
except FileNotFoundError:
    # Try alternative file
    df = pd.read_csv('../data files/austin_housing_cleaned.csv')
    print(f"Loaded austin_housing_cleaned.csv with {len(df)} records")

# Display basic information
print(f"\nColumns available: {list(df.columns)}")
print(f"Year range: {df['latest_saleyear'].min()} - {df['latest_saleyear'].max()}")
print(f"Number of ZIP codes: {df['zipcode'].nunique()}")
print(f"\nFirst few rows:")
df[['zipcode', 'latitude', 'longitude', 'latestPrice', 'latest_saleyear']].head()

Loaded data.csv with 15171 records

Columns available: ['zpid', 'city', 'streetAddress', 'zipcode', 'description', 'latitude', 'longitude', 'propertyTaxRate', 'garageSpaces', 'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa', 'hasView', 'homeType', 'parkingSpaces', 'yearBuilt', 'latestPrice', 'numPriceChanges', 'latest_saledate', 'latest_salemonth', 'latest_saleyear', 'latestPriceSource', 'numOfPhotos', 'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures', 'numOfWaterfrontFeatures', 'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'homeImage']
Year range: 2018 - 2021
Number of ZIP codes: 48

First few rows:


,zipcode,latitude,longitude,latestPrice,latest_saleyear
0,78660,30.430632,-97.663078,305000,2019
1,78660,30.432672,-97.661697,295000,2020
2,78660,30.409748,-97.639771,256125,2019
3,78660,30.432112,-97.661659,240000,2018
4,78660,30.437368,-97.656860,239900,2018


In [5]:
# Data cleaning and preparation
# Remove rows with missing critical data
df_clean = df[['zipcode', 'latitude', 'longitude', 'latestPrice', 'latest_saleyear']].copy()
df_clean = df_clean.dropna(subset=['zipcode', 'latestPrice', 'latest_saleyear'])

# Ensure proper data types
df_clean['zipcode'] = df_clean['zipcode'].astype(str)
df_clean['latest_saleyear'] = df_clean['latest_saleyear'].astype(int)
df_clean['latestPrice'] = df_clean['latestPrice'].astype(float)

# Filter out outliers (prices must be positive and reasonable)
df_clean = df_clean[(df_clean['latestPrice'] > 0) & (df_clean['latestPrice'] < 10000000)]

print(f"Cleaned data shape: {df_clean.shape}")
print(f"Years available: {sorted(df_clean['latest_saleyear'].unique())}")
print(f"ZIP codes: {df_clean['zipcode'].nunique()}")

Cleaned data shape: (15168, 5)
Years available: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021)]
ZIP codes: 48


In [6]:
# Calculate median housing prices by ZIP code and year
median_by_zip_year = df_clean.groupby(['zipcode', 'latest_saleyear']).agg({
    'latestPrice': 'median',
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

# Rename columns for clarity
median_by_zip_year.columns = ['zipcode', 'year', 'median_price', 'latitude', 'longitude']

# Calculate year-over-year price change
median_by_zip_year = median_by_zip_year.sort_values(['zipcode', 'year'])
median_by_zip_year['prev_year_price'] = median_by_zip_year.groupby('zipcode')['median_price'].shift(1)
median_by_zip_year['yoy_change'] = ((median_by_zip_year['median_price'] - median_by_zip_year['prev_year_price']) / 
                                      median_by_zip_year['prev_year_price'] * 100)
median_by_zip_year['yoy_change'] = median_by_zip_year['yoy_change'].fillna(0)

# Format for display
median_by_zip_year['median_price_fmt'] = median_by_zip_year['median_price'].apply(lambda x: f'${x:,.0f}')
median_by_zip_year['yoy_change_fmt'] = median_by_zip_year['yoy_change'].apply(
    lambda x: f'+{x:.1f}%' if x > 0 else f'{x:.1f}%' if x < 0 else 'N/A'
)

print(f"\nAggregated data shape: {median_by_zip_year.shape}")
print(f"\nSample of aggregated data:")
median_by_zip_year.head(10)


Aggregated data shape: (171, 9)

Sample of aggregated data:


,zipcode,year,median_price,latitude,longitude,prev_year_price,yoy_change,median_price_fmt,yoy_change_fmt
0,78617,2018,194900.0,30.166473,-97.633489,NaN,0.000000,"$194,900",N/A
1,78617,2019,205000.0,30.163992,-97.634620,194900.0,5.182145,"$205,000",+5.2%
2,78617,2020,200000.0,30.162966,-97.633781,205000.0,-2.439024,"$200,000",-2.4%
3,78617,2021,137500.0,30.160576,-97.639084,200000.0,-31.250000,"$137,500",-31.2%
4,78619,2018,650000.0,30.139151,-97.972824,NaN,0.000000,"$650,000",N/A
5,78619,2019,719495.0,30.138452,-97.974190,650000.0,10.691538,"$719,495",+10.7%
6,78619,2020,720000.0,30.133125,-97.978281,719495.0,0.070188,"$720,000",+0.1%
7,78652,2020,360929.5,30.147471,-97.846363,NaN,0.000000,"$360,930",N/A
8,78653,2019,369900.0,30.330337,-97.601929,NaN,0.000000,"$369,900",N/A
9,78653,2020,297490.0,30.366128,-97.605877,369900.0,-19.575561,"$297,490",-19.6%


In [7]:
# Identify key insights for annotations
# Find ZIP codes with highest growth
growth_by_zip = median_by_zip_year.groupby('zipcode').agg({
    'yoy_change': 'mean',
    'median_price': 'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

# Get top growth areas
top_growth_zip = growth_by_zip.nlargest(3, 'yoy_change')
print("Top 3 ZIP codes by average YoY growth:")
print(top_growth_zip[['zipcode', 'yoy_change', 'median_price']])

# Identify East Austin ZIP codes (typically 787xx range)
east_austin_zips = median_by_zip_year[median_by_zip_year['zipcode'].str.startswith('787')]
if len(east_austin_zips) > 0:
    avg_growth_east = east_austin_zips['yoy_change'].mean()
    print(f"\nEast Austin (787xx) average YoY growth: {avg_growth_east:.2f}%")

Top 3 ZIP codes by average YoY growth:
   zipcode  yoy_change  median_price
5    78701  180.126720  1.678667e+06
9    78705   24.559938  5.864333e+05
12   78721   22.600990  4.162250e+05

East Austin (787xx) average YoY growth: 10.00%


In [8]:
# Create the choropleth map with animation
fig = px.choropleth_mapbox(
    median_by_zip_year,
    geojson=None,
    locations='zipcode',
    color='median_price',
    animation_frame='year',
    hover_name='zipcode',
    hover_data={
        'zipcode': True,
        'median_price_fmt': True,
        'yoy_change_fmt': True,
        'year': True,
        'median_price': False,
        'latitude': False,
        'longitude': False
    },
    color_continuous_scale='Viridis',
    mapbox_style='carto-positron',
    zoom=9,
    center={'lat': median_by_zip_year['latitude'].mean(), 
            'lon': median_by_zip_year['longitude'].mean()},
    opacity=0.6,
    labels={
        'median_price': 'Median Home Price ($)',
        'median_price_fmt': 'Median Price',
        'yoy_change_fmt': 'YoY Change',
        'zipcode': 'ZIP Code',
        'year': 'Year'
    }
)

# Update layout for better visualization
fig.update_layout(
    title={
        'text': 'Geographic Distribution of Median Housing Prices in Austin, TX',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'color': '#333333'}
    },
    coloraxis_colorbar={
        'title': {
            'text': 'Median Home Price ($)',
            'side': 'bottom'
        },
        'thickness': 20,
        'len': 0.7,
        'x': 0.5,
        'xanchor': 'center',
        'y': -0.15,
        'yanchor': 'bottom',
        'orientation': 'h'
    },
    height=700,
    margin={'r': 0, 't': 80, 'l': 0, 'b': 100}
)

# Configure animation settings
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

print("Choropleth map created successfully!")
print("Note: Using scatter mapbox visualization since we don't have GeoJSON boundaries for ZIP codes.")
print("The map shows data points for each ZIP code with animation across years.")

Choropleth map created successfully!
Note: Using scatter mapbox visualization since we don't have GeoJSON boundaries for ZIP codes.
The map shows data points for each ZIP code with animation across years.


In [9]:
# Create an alternative scatter mapbox visualization
# This works better without GeoJSON data for ZIP boundaries
fig = px.scatter_mapbox(
    median_by_zip_year,
    lat='latitude',
    lon='longitude',
    size='median_price',
    color='median_price',
    animation_frame='year',
    hover_name='zipcode',
    hover_data={
        'zipcode': True,
        'median_price_fmt': True,
        'yoy_change_fmt': True,
        'year': True,
        'median_price': False,
        'latitude': False,
        'longitude': False
    },
    color_continuous_scale='Viridis',
    size_max=40,
    zoom=9.5,
    mapbox_style='carto-positron',
    labels={
        'median_price': 'Median Home Price ($)',
        'median_price_fmt': 'Median Price',
        'yoy_change_fmt': 'YoY Change',
        'zipcode': 'ZIP Code',
        'year': 'Year'
    }
)

# Update layout
fig.update_layout(
    title={
        'text': 'Geographic Distribution of Median Housing Prices in Austin, TX<br><sub>Circle size and color represent median home prices by ZIP code</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#333333'}
    },
    coloraxis_colorbar={
        'title': {
            'text': 'Median Home Price ($)',
            'side': 'bottom'
        },
        'thickness': 20,
        'len': 0.7,
        'x': 0.5,
        'xanchor': 'center',
        'y': -0.15,
        'yanchor': 'bottom',
        'orientation': 'h'
    },
    height=700,
    margin={'r': 0, 't': 100, 'l': 0, 'b': 100}
)

# Add annotation for key insight
fig.add_annotation(
    text='Higher prices concentrated in West and Central Austin<br>East Austin shows significant growth trends',
    xref='paper',
    yref='paper',
    x=0.02,
    y=0.98,
    showarrow=False,
    bgcolor='rgba(255, 255, 255, 0.8)',
    bordercolor='#333333',
    borderwidth=1,
    borderpad=8,
    font={'size': 11, 'color': '#333333'},
    align='left',
    xanchor='left',
    yanchor='top'
)

# Configure animation settings
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

# Show the interactive map
fig.show()